In [10]:
#@title Huggingface Login
#@markdown huggingface weight 를 이용하고 싶다면 로그인 필수
from google.colab import userdata
import os

os.environ['HF_WRITE_TOKEN'] = userdata.get('HF_WRITE_TOKEN')

!huggingface-cli login --add-to-git-credential --token $HF_WRITE_TOKEN


Token is valid (permission: write).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in case you want to set the 'store' credential helper as default.

git config --global credential.helper store

Read https://git-scm.com/book/en/v2/Git-Tools-Credential-Storage for more details.
Token has not been saved to git credential helper.
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [17]:
%%writefile requirements.txt
trl
deepspeed
lightning
datasets
tokenizers
huggingface_hub
openlm

Overwriting requirements.txt


In [18]:
%%capture
!git clone https://github.com/cg123/mergekit.git
!cd mergekit && pip install -q -e .
!pip install requriements.txt

In [26]:
import yaml

MODEL_NAME = "Mistral-Nemo-Merge"

yaml_config = """
slices:
  - sources:
      - model: mistralai/Mistral-Nemo-Instruct-2407
        layer_range: [0, 16]
      - model: meta-llama/Meta-Llama-3-8B-Instruct
        layer_range: [0, 16]
      - model: google/gemma-2-9b-it
        layer_range: [0, 16]
      - model: tiiuae/falcon-mamba-7b
        layer_range: [0, 16]
      - model: Qwen/Qwen2-7B-Instruct
        layer_range: [0, 16]
      - model: microsoft/Phi-3-mini-4k-instruct
        layer_range: [0, 16]
      - model: meta-llama/Llama-2-7b-chat-hf
        layer_range: [0, 16]
merge_method: slerp
base_model: mistralai/Mistral-Nemo-Instruct-2407
parameters:
  t:
    - filter: self_attn
      value: [0, 0.5, 0.3, 0.7, 1]
    - filter: mlp
      value: [1, 0.5, 0.7, 0.3, 0]
    - value: 0.5
dtype: bfloat16
"""

# Save config as yaml file
with open('config.yaml', 'w', encoding="utf-8") as f:
    f.write(yaml_config)


In [27]:
# Merge models
!mergekit-yaml config.yaml merge --copy-tokenizer --allow-crimes --out-shard-size 1B --lazy-unpickle


스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
model-00003-of-00004.safetensors:  18% 870M/4.92G [01:13<1:24:14, 800kB/s]




model-00001-of-00004.safetensors:  18% 891M/4.98G [01:13<1:15:23, 903kB/s]

model-00004-of-00004.safetensors:  80% 933M/1.17G [01:13<03:27, 1.13MB/s]





model-00002-of-00004.safetensors:  18% 891M/5.00G [01:13<1:16:57, 890kB/s]


model-00003-of-00004.safetensors:  18% 881M/4.92G [01:13<59:29, 1.13MB/s] 




model-00001-of-00004.safetensors:  18% 902M/4.98G [01:13<55:17, 1.23MB/s] 

model-00004-of-00004.safetensors:  81% 944M/1.17G [01:13<02:20, 1.60MB/s]





model-00002-of-00004.safetensors:  18% 902M/5.00G [01:13<54:16, 1.26MB/s] 


model-00003-of-00004.safetensors:  18% 891M/4.92G [01:13<42:08, 1.59MB/s]




model-00001-of-00004.safetensors:  18% 912M/4.98G [01:13<40:08, 1.69MB/s]

model-00004-of-00004.safetensors:  82% 954M/1.17G [01:13<01:35, 2.25MB/s]





model-00002-of-00004.safetensors:  18% 912M/5.00G [01:14<38:36, 1.76MB/s]


model-00003-of-00004.safetensors: 

In [8]:
from huggingface_hub import ModelCard, ModelCardData
from jinja2 import Template

username = "mlabonne"

template_text = """
---
license: apache-2.0
tags:
- merge
- mergekit
- lazymergekit
{%- for model in models %}
- {{ model }}
{%- endfor %}
---

# {{ model_name }}

{{ model_name }} is a merge of the following models using [mergekit](https://github.com/cg123/mergekit):

{%- for model in models %}
* [{{ model }}](https://huggingface.co/{{ model }})
{%- endfor %}

## 🧩 Configuration

\```yaml
{{- yaml_config -}}
\```
"""

# Create a Jinja template object
jinja_template = Template(template_text.strip())

# Get list of models from config
data = yaml.safe_load(yaml_config)
if "models" in data:
    models = [data["models"][i]["model"] for i in range(len(data["models"])) if "parameters" in data["models"][i]]
elif "parameters" in data:
    models = [data["slices"][0]["sources"][i]["model"] for i in range(len(data["slices"][0]["sources"]))]
elif "slices" in data:
    models = [data["slices"][i]["sources"][0]["model"] for i in range(len(data["slices"]))]
else:
    raise Exception("No models or slices found in yaml config")

# Fill the template
content = jinja_template.render(
    model_name=MODEL_NAME,
    models=models,
    yaml_config=yaml_config,
    username=username,
)

# Save the model card
card = ModelCard(content)
card.save('merge/README.md')


In [11]:
from google.colab import userdata
from huggingface_hub import HfApi
import os

username = "Gunulhona"

# Defined in the secrets tab in Google Colab
api = HfApi(token=os.environ['HF_WRITE_TOKEN'])

api.create_repo(
    repo_id=f"{username}/{MODEL_NAME}",
    repo_type="model"
)
api.upload_folder(
    repo_id=f"{username}/{MODEL_NAME}",
    folder_path="merge",
)


CommitInfo(commit_url='https://huggingface.co/Gunulhona/Marcoro14-7B-slerp/commit/c33fc0ba0108bd5cecba413a18590dde0eda2f92', commit_message='Upload folder using huggingface_hub', commit_description='', oid='c33fc0ba0108bd5cecba413a18590dde0eda2f92', pr_url=None, pr_revision=None, pr_num=None)

In [12]:
from transformers import AutoTokenizer
import transformers
import torch

model = "Gunulhona/Marcoro14-7B-slerp"
messages = [{"role": "user", "content": "What is a large language model?"}]

tokenizer = AutoTokenizer.from_pretrained(model)
prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    torch_dtype=torch.float16,
    device_map="auto",
)

outputs = pipeline(prompt, max_new_tokens=256, do_sample=True, temperature=0.7, top_k=50, top_p=0.95)
print(outputs[0]["generated_text"])


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


OSError: Gunulhona/Marcoro14-7B-slerp does not appear to have a file named config.json. Checkout 'https://huggingface.co/Gunulhona/Marcoro14-7B-slerp/tree/main' for available files.